In [ ]:
import numpy as np
import pandas as pd

all_systems_csv_path = "/Users/iyngkarrankumar/Documents/Misc/Tracking models/data/all_systems.csv"
notable_models_csv_path = "data/notable_ai_models.csv"
large_scale_models_csv_path = "data/large_scale_ai_models.csv"

In [ ]:

all_systems_csv_path = "/Users/iyngkarrankumar/Documents/Misc/Tracking models/data/all_systems.csv"
notable_models_csv_path = "data/notable_ai_models.csv"
large_scale_models_csv_path = "data/large_scale_ai_models.csv"

ALL_SYSTEMS_DATA = pd.read_csv(all_systems_csv_path)
NOTABLE_SYSTEMS_DATA = pd.read_csv(notable_models_csv_path)
LARGE_SCALE_DATA = pd.read_csv(large_scale_models_csv_path)

AS_nan_ratio = (ALL_SYSTEMS_DATA['Training compute (FLOP)'].isna().sum())/len(ALL_SYSTEMS_DATA)
NS_nan_ratio = (NOTABLE_SYSTEMS_DATA['Training compute (FLOP)'].isna().sum())/len(NOTABLE_SYSTEMS_DATA)
LS_nan_ratio = (LARGE_SCALE_DATA['Training compute (FLOP)'].isna().sum())/len(LARGE_SCALE_DATA)


print(AS_nan_ratio,NS_nan_ratio,LS_nan_ratio)